In [ ]:
knitr::opts_chunk$set(echo = TRUE)



***
<style>
div.blue { background-color:#e6f0ff; border-radius: 5px; padding: 20px;}
</style>
<div class = "blue">

**Note:** If you want to look at the full R code used to solve this business case, please click on the **"Code"** button which when is displayed or go directly to my GitHub Repository by clicking [**here**](https://github.com/leoncasba/Customer_Segmentation_ecommerce).

</div>

***
# **1. Introduction**
***

Customer segmentation is the process of separating customers into subgroups on the basis of their shared behavior or other attributes. The overall aim of this process is to identify a **high-value customer base**, i.e. customers that have the highest growth potential or are the most profitable.

<center>
![](C:/Users/Leo/Desktop/Portfiolo/Customer_Segmentation_ecommerce/segmentation.gif)
</center>

The customer segmentation process allows a company to achieve many things. The most common application is to allow marketers to target marketing campaigns at specific customers to achieve a given goal, such as reactivating lapsed customers or encouraging customers to purchase an additional item.

***
# **2. Business Understanding**
***
## 2.1. Case Statement
***

>Retail is the process of selling consumer goods or services to customers through multiple channels of distribution to earn a profit. In this context, *Congos* is an e-commerce retail startup based in Costa Rica. They are specialized in selling electronics and home appliances like beds, couches, laptops, smart TVs, to mention a few. 

>Business Development and Marketing Teams are creating new strategies to increase substantially company's income as they are getting more investment in order to keep growing and make bigger its product catalog. 

>There is data from over 23,000 *Congos'* transactions made during 2018 and 2019, its first two years of acting in the e-commerce market. This valuable asset (raw data) can be converted into a powerful knowledge resource by making a customer segmentation analysis which allows who to target its promotions, sales and marketing efforts.

***
## 2.2. Objective
***

>The **objective** of this analysis is to model and differentiate the characteristics and typologies of *Congos'* customers, providing a potent asset to Business Development and Marketing teams aimed to increase its efficiency by directing different specific strategies towards the designated segments.

***
## 2.3. Business Benefit
***

>To help Business Development and Marketing Teams to develop tailor-made marketing campaigns and differentiation strategies based on the characteristics of each customer and consequently making more profitable the company by increasing sales.

***
## 2.4. Scope
***

>All the insights come strictly from the data and not from subjective opinion. However, we should have opinion from a Subject Matter Expert (SME) to get better understanding of marketing strategies and call to action.

***
## 2.5. Key Business Questions
***

  >* How recent and frequent do the customers buy? How much money do the customers have spent since their first purchase? 
  * What segments or customer's tipologies does the company have?
  * What are the recommendations to target each customer segment?

***
## 2.6. Expected Outcomes
***

>To understand the customer segmentation analysis by characterizing each segment and provide recommendations based on segment profile.

***
# 3. **Methodology & Analytic Techniques Used**
***

  * Data understanding.
  * Data cleaning and preparation.
  * Exploratory data analysis (EDA).
  * RFM Analysis.
  * Unsupervised Learning: K-means Clustering.
    


In [ ]:
#These are the R packages used for the analysis.
library(readxl)
library(knitr)
library(ggplot2)
library(dplyr)
library(tidyr)
library(lubridate)
library(clusterSim)
library(factoextra)


***
## 3.1. Data Understanding
***

  * Data consists in Congos' transactions made by customers during the period of 2018-04-01 and 2019-09-28.
  * Data was provided during a Data Analytics course. You can download it [**here,**](https://github.com/leoncasba/Customer_Segmentation_ecommerce/blob/main/Customer_Segmentation.xlsx) directly from my GitHub repository.
  * The data set has 11 columns and 23208 rows.
  * Data dictionary:
  
    * **Transaction:** transaction unique number identifier.
    * **Year:** year transactions has been made.
    * **Month:** month transactions has been made.
    * **Day:** day transactions has been made.
    * **Customer_Code:** customer's unique identification code.
    * **Province:** province where customers come from.
    * **Age:** customer's age.
    * **Product_Code:** product unique identifier code.
    * **Product_Name:** product name.
    * **Product_Price:** product price per unit ($).
    * **Quantity:** number of product units bought in each transaction. 
    
 


In [ ]:
#Read database
data <- read_excel("Customer_Segmentation.xlsx")

#Head 5 first rows from database
kable(head(data,5),  caption="**Table 1.** First five rows from Congos database.")


***

## 3.2. Data Cleaning and Preparation.

***

Findings and actions:

  * There are not missing values on the database.
  * There are not outliers considerable to remove on the data base.
  * Having a complete transaction date would be easier and better to manipulate for the analysis, so I proceed creating a feature called "Transaction Date".
  * Some features are no longer suitable for the analysis: *Year*, *Month* and *Day* (Transaction_Date has been created). I proceed eliminating those features.
  


In [ ]:
#Function to get percentage of missing values
NAporcent <- function(x, ndec=2){
  porcent=(sum(is.na(x))/length(x))*100
  p2 = round(porcent, digits=ndec)
}

#Look if there are missing values
print(paste("There are", sum(apply(data, 2, NAporcent)), "missing values."))

#Outliers
ggplot(stack(data[,-c(1,3,6,9)]), aes(x = ind, y = values,fill=ind)) + 
  geom_boxplot() +
  theme(legend.position="none") +
  scale_fill_viridis_d() +
  ggtitle("")+
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5),axis.title = element_blank())+scale_y_continuous(trans='log10')


In [ ]:
#Create TransactionDate
data$TransactionDate = ymd(paste0(data$Year,"-",data$Month,"-",data$Day))

#Eliminating features no longer necessary in the analysis
data_prepared <- data[ , -c(2, 3, 4)]  


***

## 3.3. Exploratory Data Analysis

***

  * **How much has been sold every month by year??**
  
As we can see on Figure 2, revenue differs by year but basically follows a similar pattern. Sales peak occurred during July for both years, being 2018 more successful, over passing the 800k dollars, probably product of some kind of promotion occurred during this month. Also is interesting to notice there is a gap between years on May of around 50k dollars, being more successful May 2019. Business team can replicate the success of sales strategies in this month to be implemented in further campaigns.
  


In [ ]:
q1 <- data_prepared %>% group_by(year(TransactionDate), month(TransactionDate)) %>% summarize(total=sum(Product_Price*Quantity)) %>% rename(Year='year(TransactionDate)', Month='month(TransactionDate)', Total=total)

ggplot(q1, aes(x=factor(Month), y=(Total/1000), group=factor(Year), colour=factor(Year)))+geom_line(size=1.5)+ geom_point(size=3) +
  labs(y="Total revenue ($ thousands)", x="Month", colour="Year") +theme_classic(base_size = 18)


  * **Which are the top 10 products by total revenue?**
  
Figure 3 shows top 10 products by total revenue. Highlighted in red is the top product: Laptops. It has reached a total revenue over 3 million dollars. This simple visualization have shown which are the more profitable products in the company so far, which represents a good focus to start redirecting product marketing campaigns to electronic consumers after knowing the segmentation.
  


In [ ]:
q3 <- data_prepared %>% group_by(Product_Name) %>% summarise(Total_Revenue_by_Product=sum(Product_Price*Quantity)/1000) %>% arrange(desc(Total_Revenue_by_Product)) %>% top_n(10)

ggplot(q3, aes(x=reorder(factor(Product_Name),Total_Revenue_by_Product), y=Total_Revenue_by_Product, fill = ifelse(Product_Name == "Laptop", "Highlighted", "Normal") ))+geom_col()+theme_classic(base_size = 18)+coord_flip()+theme(legend.position = "none", axis.title.y = element_blank())+labs(y="Total revenue by product ($ thousands)")


  
  * **How is customer's age distributed?**  
  
As we can see on Figure 4 age is uniformly distributed. Having a range between 27 and 53 years old, that is the age BD & Marketing should focus to target.
  


In [ ]:
ggplot(data_prepared, aes(Age))+geom_histogram(aes(y = ..density..), colour = 1, fill = "white")+geom_density(lwd = 1, colour = 4,
               fill = 4, alpha = 0.25)+theme_classic(base_size = 18)+xlim(25, 58)


***
  
## 3.4. RFM Analysis

***

Recency Frequency Monetary (RFM) analysis allows to segment customers by the frequency and value of purchases and identify those customers who spend the most money.

  * **Recency** means how long it’s been since a customer bought any product (days).
  * **Frequency** represents how often a customer buys (number of purchases in a period of time).
  * **Monetary value** is the total value of purchases a customer has made (total amount in dollars).
  


In [ ]:
NOW=max(data_prepared$TransactionDate)
#The last transaction date is 2019-12-28, so I will use this date to calculate Recency.

#Recency-Frequency-Monetary
RFM <- data_prepared %>% group_by(Customer_Code) %>% summarise(Recency=as.numeric(NOW-max(TransactionDate)), Frequency=n(), Monetary=sum(Quantity*Product_Price))


Once is done the RFM table, I split the metrics into segments using quantiles. 



In [ ]:
quantile_table <- (apply(RFM[,-1] , 2, quantile))
kable(quantile_table,caption="**Table 2.** RFM table - Quantiles.")


I assigned a quantile "score" from 1 to 4 to each *Recency*, *Frequency* and *Monetary* respectively, regarding its position:

  * 4 is the highest value (customers is on the top of the evaluated value), and 1 is the lowest value (customer is on the bottom of the evaluated value).
  * A final RFM score (Overall Value) is calculated simply by concatenating individual RFM score numbers:
<br><br>
$$RFM \ Score = R_{quartile} + F_{quartile} + M_{quartile}$$


In [ ]:
quantiles <- apply(RFM[,-1] , 2, ntile, n=4)
colnames(quantiles) <- c("R_quartile",	"F_quartile",	"M_quartile")
RFM_q <- cbind(RFM, quantiles)
RFM_q$RFM_score <- paste0(as.character(RFM_q$R_quartile),as.character(RFM_q$F_quartile),as.character(RFM_q$M_quartile))
kable(head(RFM_q,5),  caption="**Table 3.** RFM scores table - Sample of five customers.")


The table below (adapted from [**here.**](https://runawayhorse001.github.io/LearningApacheSpark/rfm.html)) show which means each segment and its respective marketing strategy according with its RFM score:

<center>
![](C:/Users/Leo/Desktop/Portfiolo/Customer_Segmentation_ecommerce/RFM_table.JPG)
</center>


Now it is possible to categorize each customer by its respective segment. Figure 5 shows number of customers by segment, however, "Others" category represents most of the customers (607), which was not showed on the plot to focus on the customers truly categorized. Those uncategorized will be approached in the next step of the analysis by performing a segmentation with K-means. In the mean time let's focus on figure below which provide an overview of what is the distribution of our customers properly categorized on the different segments displayed.


In [ ]:
#Set each level to customers

RFM_q$Category <-
  ifelse(RFM_q$RFM_score == '144', "Best Customers",
    ifelse(RFM_q$F_quartile == '4', "Loyal Customers",
      ifelse(RFM_q$M_quartile == '4', "Big Spenders", 
        ifelse(RFM_q$RFM_score == '333' | RFM_q$RFM_score == '444' | RFM_q$RFM_score == '433' , "Almost Lost", 
          ifelse(RFM_q$RFM_score == '411', "Lost Cheap Customers", "Others")))))

group <- RFM_q %>% group_by(Category) %>% summarize(N_Customers=n()) 
group$Category <- factor(group$Category,                                   
                  levels = group$Category[order(group$N_Customers, decreasing = TRUE)])

ggplot(group[-6,], aes(x=Category, y=N_Customers))+geom_col(fill = "#0073C2FF")+theme_classic(base_size = 18)+labs(y="Number of Customers", x="Segment")+geom_text(aes(label = N_Customers), vjust = 2, cex=6, color = "white")


***

## 3.5. K-means Clustering

***

From previous RFM analysis we assume there are about 5 subgroup of customers, however, there is a big amount of customers who remains in the "Other" category. 

K-means comes to provide a solution by grouping all the customers in their respective subgroups according with their homogeneous properties. This algorithm is an unsupervised machine learning method used to find homogeneous subgroups within a population. 

To process the learning data, the K-means algorithm starts with a first group of randomly selected centroids, which are used as the beginning points for every cluster, and then performs iterative (repetitive) calculations to optimize the positions of the centroids. It halts creating and optimizing clusters when either:

  * The centroids have stabilized — there is no change in their values because the clustering has been successful.
  * The defined number of iterations has been achieved.

For further information about how the algorithm works you can check this useful [**article**](https://towardsdatascience.com/k-means-clustering-explained-4528df86a120).

***

### 3.5.1 Data pre-processing

***

Before proceed with the K-Means algorithm, data preparation is required in order to obtain a good performance on the modeling, for hence, the input data requires:

  * **No outliers.**
  
As we can see Frequency and Monetary variables have values over the interquartile range, however, nothing to be worried about, it seems like there is someone who represents a huge amount of monetary amount as well as several customers which buy really frequently. In the other side, recency shows multiple values out of the interquartile range, it seems like data distribution is skewed more than the existance of outliers, for hence, I will continue with the assumptions.
  


In [ ]:
#Checking outliers
pivot <- RFM %>% pivot_longer(Recency:Monetary)
ggplot(data = pivot, aes(x=name, y=value)) + 
             geom_boxplot(fill = "#0073C2FF")+facet_wrap( ~ name, scales="free")+theme_classic(base_size = 18)+labs(y="Value", x="Variables (R-F-M)")


  * **Data has symmetric distribution of variables (it isn’t skewed).**
  
As we can see on the figures below Frequency and Monetary Value have normal distribution, however recency is right-skewed as I assumed on the previous analysis. This means, data will need a transformation in order to use on further K-means algorithm.


In [ ]:
#Check distribution of each variable
ggplot(RFM, aes(Recency))+geom_histogram(aes(y = ..density..), colour = 1, fill = "white")+geom_density(lwd = 1, colour = 4,
               fill = 4, alpha = 0.25)+theme_classic(base_size = 18)+ggtitle("Recency distribution")

ggplot(RFM, aes(Frequency))+geom_histogram(aes(y = ..density..), colour = 1, fill = "white")+geom_density(lwd = 1, colour = 4,
               fill = 4, alpha = 0.25)+theme_classic(base_size = 18)+ggtitle("Frequency distribution")

ggplot(RFM, aes(Monetary))+geom_histogram(aes(y = ..density..), colour = 1, fill = "white")+geom_density(lwd = 1, colour = 4,
               fill = 4, alpha = 0.25)+theme_classic(base_size = 18)+ggtitle("Monetary Value distribution")


I transformed recency data using a natural algorithm and added a small constant as log transformation demands all the values to be positive.



In [ ]:
#Checking distribution on log transformed recency data. 
ggplot(RFM, aes(log10(Recency+1)))+geom_histogram(aes(y = ..density..), colour = 1, fill = "white")+geom_density(lwd = 1, colour = 4,
               fill = 4, alpha = 0.25)+theme_classic(base_size = 18)+ggtitle("Recency (transformed) distribution")

#Creating matrix for next step: scaling
RFM_2 <- cbind(RFM[, c(3,4)], (log10(RFM$Recency+1)))


  * **Variables are on the same scale.**
  
Since K-means algorithm works with euclidean distances we need to escalate the data. It means transforming data input features in order the mean is equal 0 and the standard deviation equal 1.


In [ ]:
#Scale values
scaled_values <- apply(RFM_2, 2, scale, center = TRUE, scale = TRUE)

#Check if are correctly scaled.
paste("Scaled values of Recency, Frequency and Monetary value have mean equal", mean(round(apply(scaled_values, 2, mean))), "and standard deviation equal", mean(apply(scaled_values, 2, sd)))


***

### 3.5.2. Selecting the optimum K

***

The process of accessing the quality of clustering can be a tricky task. For that reason, I tried three different measures to determine the quality of clustering: Elbow Method, Davies-Bouldin Index and Silhouette Analysis. You can dive deep about how each of this measures work in [**this useful article.**](https://gdcoder.com/silhouette-analysis-vs-elbow-method-vs-davies-bouldin-index-selecting-the-optimal-number-of-clusters-for-kmeans-clustering/)


***

#### Elbow method

***

Idea behind the elbow method is to identify the value of k where the score begins to decrease most rapidly before the curve reached a plateau. In the figure below, we can see it seems is around 3 or 4, however, I will explore further on the next measures.


In [ ]:
# Initializing total within sum of squares error: wss
wss <- 0

# For 1 to 10 cluster centers, save total within sum of squares to wss variable
for (i in 1:10) {
  test <- kmeans(scaled_values, centers = i, nstart = 20)
  # 
  wss[i] <- test$tot.withinss
}

# Plot total within sum of squares vs. number of clusters
plot(1:10, wss, type = "b", 
     xlab = "Number of Clusters", 
     ylab = "Within groups sum of squares")


***

#### Davies-Bouldin Index

***

The minimum score is zero and the optimal clustering has the smallest Davies–Bouldin index value. As we can see on Table 4, the lowest Davies-Bouldin Index corresponds to k=4.


In [ ]:
#First we set the random seed
set.seed(1995)
#I calculate kmeans using k values from 3 to 6
lst1 <- lapply(3:6, function(i) kmeans(scaled_values, centers=i, nstart = 20))
names(lst1) <- paste0("cluster_", 3:6)
#Store cluster results in a tibble
k_ <- as_tibble(scaled_values) %>% mutate(k3=unlist(lst1$cluster_3[1]), k4=unlist(lst1$cluster_4[1]), k5=unlist(lst1$cluster_5[1]), k6=unlist(lst1$cluster_6[1])) 
#Calculate Davies-Boulding Index
index_DB <- lapply(4:7, function(i) index.DB(k_, k_[,i], d=NULL, centrotypes="centroids", p=2, q=2))
names(index_DB) <- paste0("index_cluster_k", 3:6)
#Comparing DB Index for each k tried
DB_index <- rbind(index_DB$index_cluster_k3[1], index_DB$index_cluster_k4[1], index_DB$index_cluster_k5[1], index_DB$index_cluster_k6[1])
DB_index_ <- as_tibble(unlist(DB_index))
DB_index_$k <- c(3, 4, 5, 6)
kable(DB_index_, digits=2, col.names=c("DB index", "Number of clusters"), caption="**Table 4.** Davies-Bouldin Index Values.")


***

#### Silhouette Analysis

***

When using the the silhouette analysis we are normally looking for a value of k that provides the high average silhouette score, in this case the figure shows is k=2. However, I will follow to logic presented by the previous two metrics which make more sense on the context of the problem I'm solving.


In [ ]:
fviz_nbclust(scaled_values, pam, method = "silhouette")+ theme_classic()



***

## 3.6. K-means Results & Interpretation

***

I defined k=4 as the optimum number of clusters and applied the algorithm to the data, getting the results below:


In [ ]:
set.seed(1995)
k_definitive <- kmeans(scaled_values, centers=4, nstart = 20)

fviz_cluster(k_definitive, data = scaled_values,
             palette = c("#2E9FDF", "#00AFBB", "#E7B800", "#FF0000"), 
             geom = "point",
             ellipse.type = "convex", 
             ggtheme = theme_classic(base_size = 18),
             main = ""
             )


Now the clusters are defined it is time to label the customers with its corresponding cluster. For that I use the mean to evaluate each cluster and provide an interpretation of the clusters formed using k-means.



In [ ]:
# Extracting the vector of cluster assignments from the model
clust_customers <- k_definitive$cluster
# Building the segment_customers data frame
RFM_k_clusters <- mutate(RFM, Cluster = clust_customers)
# Calculating the size of each cluster
count <- count(RFM_k_clusters, Cluster)
# Calculating the mean for each category+adding count column
RFM_final <- RFM_k_clusters[,-1] %>% 
  group_by(Cluster) %>% 
  summarise_all(list(mean)) %>% mutate(N=count$n, Percentage=((count$n)/sum(count$n)*100))


In [ ]:
RFM_final$Cluster <- factor(RFM_final$Cluster,                                   
                  levels = RFM_final$Cluster[order(RFM_final$N, decreasing = TRUE)])

ggplot(RFM_final, aes(x=as.factor(Cluster), y=N))+geom_col(fill = "#0073C2FF")+theme_classic(base_size = 18)+labs(y="Number of Customers", x="Cluster")+geom_text(aes(label = N), vjust = 2, cex=6, color = "white")


In [ ]:
kable(RFM_final, digits=2, caption="**Table 4.** Average RFM Measures and Proportions for each cluster.")



This is the interpretation provided to the clusters based on average metrics:

  * **Cluster 1** has 19.07% of the total customers. This cluster can be classified to the "Best Customers" segment as they purchase recently (R=1), are frequent buyers (F=4) and spent the most (M=4).

  * **Cluster 2** has 21.16% customers. It is maybe the complex segment to interpret but can be classified as "Recent but Promising Customers" These are customer who purchase recently (R=1) but do not have purchase considerably frequent (F=2) neither have spend a lot of money, but are promising ones (M=2)

  * **Cluster 3** has 30.43% of the total of customers. It can be interpreted as "Almost Lost Customers". They haven't purchased for some time (F=3) but purchased frequently (F=3) and spend good money (M=3).

  * **Cluster 4** has 29,34% of the total customers. It belongs to the "Lost Cheap Customers" segment which is characterized by their last purchase is long ago (R=4), purchased very few (F=1) and spent little money (M=1).
  

# **4. Recommendations**
***

  * **Recommendation for “Best Customers" segment:** Upsell higher value products. Ask for reviews. Engage and reward them, they can be early adopters for new products and will promote the company's brand.
  
  * **Recommendation for “Recent but Promising Customers"** segment: Create brand awareness, offer promotions. Provide on-boarding support, give them early success and start building relationship.
  
   * **Recommendation for “Almost Lost" segment:** This customer segment is very at risk for churn, so focus on activating customers and making repurchases by forming a reactivation and retention strategy. Send personalized emails to reconnect, offer renewals, provide helpful resources. We need to bring them back!
   
  * **Recommendation for “Lost Cheap Customers"** segment: This customer segment probably has churned, so the focus of the campaign is to reactivate the customer by forming a reactivation strategy. We try to revive interest, however if they does not show up is better to ignore and focus on more profitable typology of customer.

***
<center>
>This is a work made by

<center>
![](C:/Users/Leo/Documents/themes/raditian-free-hugo-theme-data-science/static/img/logo_small.png)

>*I appreciate your valuable feedback and suggestions. Go check my porfolio [**here.**](https://leon-datascience.netlify.app/)*

***
